In [ ]:
import requests
from datetime import datetime, timedelta

headers = {
    'X-Auth-Token': 'c0efea6535bc472fbc39c9f1ca96fa91'
}
url = 'http://api.football-data.org/v2/teams/86/matches?status=FINISHED'

response = requests.get(url, headers=headers)
matches = response.json()

for match in matches['matches']:
    start_time = datetime.strptime(match['utcDate'], '%Y-%m-%dT%H:%M:%SZ')
    # Assuming 90 minutes + 15 minutes for half-time break
    finish_time = start_time + timedelta(minutes=105)
    result = f"{match['homeTeam']['name']} vs {match['awayTeam']['name']} - {match['score']['fullTime']['homeTeam']}:{match['score']['fullTime']['awayTeam']}"
    winner = match['score']['winner']
    if winner == 'HOME_TEAM':
        winner = match['homeTeam']['name']
    elif winner == 'AWAY_TEAM':
        winner = match['awayTeam']['name']
    else:
        winner = "Draw"
    
    print(f"Start Time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}, Finish Time: {finish_time.strftime('%Y-%m-%d %H:%M:%S')}, Result: {result}, Winner: {winner}")


In [ ]:
#Get upcoming/past
import requests

headers = {
    'X-Auth-Token': 'c0efea6535bc472fbc39c9f1ca96fa91'
}
from datetime import datetime, timedelta

url = 'http://api.football-data.org/v2/teams/86/matches?status=SCHEDULED'

response = requests.get(url, headers=headers)
matches = response.json()


for match in matches['matches']:
    home_team = match['homeTeam']['name']
    away_team = match['awayTeam']['name']
    home_score = match['score']['fullTime']['homeTeam']
    away_score = match['score']['fullTime']['awayTeam']
    start_time = datetime.strptime(match['utcDate'], '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d %H:%M:%S')
    # End time not provided directly by API, example assumes 90 minutes + start time for simplicity
    # For accurate end time, additional data on match duration and added time would be necessary
    finished_normal_time = 'Yes' if match['status'] == 'FINISHED' else 'No'
    result = f"{home_team} vs {away_team} - {home_score}:{away_score}"
    
 
    print(f"Start Time: {start_time}, Result: {result}, Finished in Normal Time: {finished_normal_time}")
